In [ ]:
from fastapi import FastAPI, Query
from fastapi.middleware.cors import CORSMiddleware
import pandas as pd
from filters import load_data, apply_query, apply_facets, Criteria, safe_top

app = FastAPI(title="Toyota Finance API", version="1.0")

# Allow requests from your React dev server
origins = [
    "http://127.0.0.1:8000",  # or 3000, depending on React port
    "http://127.0.0.1:8000",
]
app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Load data once at startup
df_base = load_data("cars_priced.csv")

@app.get("/")
def root():
    return {"message": "Toyota Finance API running"}

@app.get("/filter")
def filter_cars(
    q: str | None = Query(None, description="Search query"),
    mpg: str = Query("any", description="low/med/high"),
    price_min: int = Query(4000),
    price_max: int = Query(60000),
    hp_min: int = Query(80),
    hp_max: int = Query(500),
    mil_min: int = Query(0),
    mil_max: int = Query(200000),
):
    crit = Criteria(
        query=q,
        mpg_bucket=mpg,
        price_range=(price_min, price_max),
        hp_range=(hp_min, hp_max),
        mileage_range=(mil_min, mil_max)
    )
    df = apply_facets(df_base, crit)
    results = safe_top(df, 25).to_dict(orient="records")
    return {"count": len(df), "results": results}

@app.get("/car/{model_name}")
def car_details(model_name: str):
    df = df_base[df_base["model"].str.contains(model_name, case=False, na=False)]
    return df.to_dict(orient="records")
